# 3. Topic Modeling

### Loading in data

In [74]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse
import nltk
#nltk.download('punkt')

from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from gensim import corpora, models, matutils
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [75]:
#!python3 -m spacy download en_core_web_sm

In [76]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [77]:
#!pip install python-Levenshtein

In [78]:
df = pd.read_pickle("df_clean.pkl")  
df.head()

,Q1 years,Q2 years,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16 rank_cat
1,NaN,NaN,NaN,0.0,None,0.0,None,None,None,None,None,0.0,None,None,None,NaN
2,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,NaN,None,None,None,3.0
3,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,NaN,None,None,None,NaN
4,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,NaN,None,None,None,NaN
5,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,NaN,None,None,None,NaN


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195 entries, 1 to 195
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1 years      177 non-null    float64
 1   Q2 years      176 non-null    float64
 2   Q3            173 non-null    object 
 3   Q4            180 non-null    float64
 4   Q5            79 non-null     object 
 5   Q6            155 non-null    float64
 6   Q7            39 non-null     object 
 7   Q8            98 non-null     object 
 8   Q9            73 non-null     object 
 9   Q10           109 non-null    object 
 10  Q11           109 non-null    object 
 11  Q12           149 non-null    float64
 12  Q13           17 non-null     object 
 13  Q14           112 non-null    object 
 14  Q15           86 non-null     object 
 15  Q16 rank_cat  152 non-null    float64
dtypes: float64(6), object(10)
memory usage: 25.9+ KB


### <b>Final Dataset Data Plan </b>   
Q1: How long have you been on [LOCATION]?  <b>Coded as YEARS</b>  
Q2: How long have you been in the [DOD SERVICE COMPONENT] (or connected to the [DOD SERVICE COMPONENT], if civilian)?  <b>Coded as YEARS</b>  
<s>Q3: What are your current responsibilities or what do you do in a typical day here?  <b>Need to explore...</s></b>  
Q4: Do you have awareness of the [CLINIC] and what it does?  <b>Coded as Yes (1), No (0)</b>  
Q5: If yes, what have you seen or heard?  <b>word cloud, Topic modeling and sentiment analysis</b>  
Q6: Do you have awareness of the prevention services offered by the [CLINIC]?  <b>Coded as Yes (1), No (0)</b>    
Q7: If yes, what have you seen or heard?  <b>word cloud, Topic modeling and sentiment analysis</b>   
Q8: In your opinion, what can [CLINIC] Prevention & Education Specialists do to support [SERVICE MEMBERS] and their families in [LOCATION]?  <b>Topic modeling</b>  
Q9: Do you have any other thoughts or suggestions?  <b>Topic modeling and sentiment analysis</b>  
Q10: What is your impression of the relationships [SERVICE MEMBERS] have with their children here in [LOCATION]? <b>Sentiment analysis</b>  
Q11:  What is your impression of the relationships [SERVICE MEMBERS] have with their significant others in [LOCATION]? <b>Sentiment analysis</b>  
Q12: Do you think there are problems here related to child abuse or domestic abuse?  <b>Coded as Yes (1), No (0)</b>  
Q13: If yes, can you tell us about those problems?   <b>Topic modeling</b>       
Q14: Can you tell me about your command's approach here related to family violence?  <b>Topic modeling and sentiment analysis</b>  
Q15: Is there anything else you'd like to comment on?  <b>Topic modeling and sentiment analysis</b>  
Q16: What is your rank/grade?  <b>Coded as Junior (1) NCO (2) SNCO (3) O1+ (4) Spouse/Dep (5) Other (6)</b>  

# 3.1 Topic Modeling Q8

<b>Q8</b>

For topic modeling, the input is 1) document term matrix, 2) number of topics, and 3) number of iterations.

Goal is to perform topic modeling for all the responses in Q8.

<i>Our goal is to find themes across the responses to Q8.</i>

In [1]:
#df_no_lem = pd.read_pickle("df_clean_no_lemma.pkl") 
#pd.set_option('display.max_colwidth', None)
#df_no_lem.groupby(by=['Q4']).Q8.unique()
#print(df_no_lem['Q8'].unique())
#98 responses
#viewing all responses grouped by awareness to CLINIC in Question #4

In [81]:
Q8_corpus = list(df['Q8'])

#195 documents, so 2/195 = .010 (which is 1%)
#words must appear at least 1% of time to show in dtm (or at least in 2 documents)
#including 1 and 2 grams
#removing stop words
#https://stackoverflow.com/questions/63479213/attributeerror-nonetype-object-has-no-attribute-lower
#https://bobbyhadz.com/blog/python-attributeerror-nonetype-object-has-no-attribute-lower#:~:text=The%20Python%20%22AttributeError%3A%20'NoneType,call%20lower()%20on%20strings.

tv = TfidfVectorizer(stop_words='english', lowercase = True, min_df=.01, ngram_range=(1,2))
x = tv.fit_transform([str(x) for x in Q8_corpus if x is not np.nan]) #make sure to only call lower(),
#which happens in tfidf vectorizer on strings and not on NONE values

Q8_tfidf = pd.DataFrame(x.toarray(), columns=tv.get_feature_names())
Q8_tfidf

/Users/nicole/opt/anaconda3/envs/unsupervised/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,advertise,advertise service,assist,check,child,clear,complaint,continue,corp,education,...,stigma,support,support family,sure,thing,think,unit,work,yes,younger
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
191,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
192,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
193,0.483242,0.511661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.483242,0.0,0.0,0.000000


In [2]:
#now we have a TERM document matrix
#easier to manipualte
#term_doc = Q8_tfidf.transpose()
#term_doc.head()
#code to help me do all the below cells:
#https://github.com/adashofdata/nlp-in-python-tutorial

### 3.1a Topic Modeling - Attempt #1 (all text minus stop words)
- To perform LDA topic modeling we need 3 things: DTM transposed, number of topics, and number of iterations.

<b>Q8</B>


In [7]:
#trying topic modeling with 2 topics
#sparse_counts = scipy.sparse.csr_matrix(term_doc)
#corpus = matutils.Sparse2Corpus(sparse_counts)
#id2word = dict((v, k) for k, v in tv.vocabulary_.items())
#lda = models.LdaModel(corpus=corpus, num_topics=2, id2word=id2word, passes=80)
#lda.print_topics()

Here are the 10 most important words for each of the 2 topics we found in above.

Two topics seem to emerge...

- Topic 0: help, family
- Topic 1: family, service

### 3.1b Topic Modeling - Attempt #2 (nouns+adjectives+noun plural & max_df & min_df


In [84]:
df_no_lem = pd.read_pickle("df_clean_no_lemma.pkl") 
df_topicmodel = df_no_lem['Q8'].dropna(axis=0)

df_topicmodel.info()


<class 'pandas.core.series.Series'>
Int64Index: 98 entries, 6 to 195
Series name: Q8
Non-Null Count  Dtype 
--------------  ----- 
98 non-null     object
dtypes: object(1)
memory usage: 1.5+ KB


In [4]:
#df_topicmodel.head()

In [86]:
def NN_JJ_NNS(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ' or pos[:2] == 'NNS'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [5]:
from nltk import word_tokenize, pos_tag

#data_NN_JJ_NNS = pd.DataFrame(df_topicmodel.apply(NN_JJ_NNS))
#data_NN_JJ_NNS

In [6]:
#Q8_corpus = list(data_NN_JJ_NNS['Q8'])
#Q8_corpus

In [8]:
# Create a new document-term matrix using only NN JJ NNP NNS

#195 documents, so 2/195 = .010 (which is 1%)
#words must appear at least 1% of time to show in dtm
#including 1 and 2 grams
#removing stop words

#tv2 = TfidfVectorizer(stop_words='english', lowercase = True, min_df=.01, ngram_range=(1,2))
#x2 = tv2.fit_transform([str(x) for x in Q8_corpus if x is not np.nan]) #make sure to only call lower(),
#which happens in tfidf vectorizer on strings and not on NONE values

#Q8_tfidf_2 = pd.DataFrame(x2.toarray(), columns=tv2.get_feature_names())
#Q8_tfidf_2


In [91]:
term_doc_2 = Q8_tfidf_2.transpose()
term_doc_2.head()

,0,1,2,3,4,5,6,7,8,9,...,88,89,90,91,92,93,94,95,96,97
abuse,0.0,0.0,0.0,0.31135,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
acclimate,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
acclimate military,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
advertisment,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
advocate,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
#trying topic modeling with 2 topics
sparse_counts = scipy.sparse.csr_matrix(term_doc_2)
corpus_2 = matutils.Sparse2Corpus(sparse_counts)
id2word_2 = dict((v, k) for k, v in tv2.vocabulary_.items())
lda_2 = models.LdaModel(corpus=corpus, num_topics=2, id2word=id2word_2, passes=80)
lda_2.print_topics()

2022-12-18 03:14:26,566 : INFO : using symmetric alpha at 0.5
2022-12-18 03:14:26,568 : INFO : using symmetric eta at 0.5
2022-12-18 03:14:26,569 : INFO : using serial LDA version on this node
2022-12-18 03:14:26,571 : INFO : running online (multi-pass) LDA training, 2 topics, 80 passes over the supplied corpus of 195 documents, updating model once every 195 documents, evaluating perplexity every 195 documents, iterating 50x with a convergence threshold of 0.001000
2022-12-18 03:14:26,618 : INFO : -7.909 per-word bound, 240.4 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:14:26,619 : INFO : PROGRESS: pass 0, at document #195/195
2022-12-18 03:14:26,687 : INFO : topic #0 (0.500): 0.034*"board" + 0.032*"available" + 0.019*"complaints" + 0.016*"campaign" + 0.012*"command" + 0.010*"continue" + 0.010*"care" + 0.009*"center military" + 0.009*"bring" + 0.009*"comment"
2022-12-18 03:14:26,687 : INFO : topic #1 (0.500): 0.028*"campaign" + 0.023*"boa

2022-12-18 03:14:27,584 : INFO : topic diff=0.004388, rho=0.301511
2022-12-18 03:14:27,637 : INFO : -5.705 per-word bound, 52.2 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:14:27,638 : INFO : PROGRESS: pass 10, at document #195/195
2022-12-18 03:14:27,663 : INFO : topic #0 (0.500): 0.052*"board" + 0.043*"available" + 0.023*"complaints" + 0.015*"continue" + 0.012*"capable" + 0.012*"command" + 0.012*"bring" + 0.011*"career goals" + 0.010*"care" + 0.010*"case incident"
2022-12-18 03:14:27,664 : INFO : topic #1 (0.500): 0.042*"campaign" + 0.020*"class learning" + 0.016*"assist paperwork" + 0.014*"child abuse" + 0.013*"care" + 0.013*"child removal" + 0.012*"center military" + 0.011*"advocate" + 0.011*"advertisment" + 0.010*"conduct check"
2022-12-18 03:14:27,665 : INFO : topic diff=0.003614, rho=0.288675
2022-12-18 03:14:27,704 : INFO : -5.704 per-word bound, 52.1 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2

2022-12-18 03:14:28,252 : INFO : topic #0 (0.500): 0.052*"board" + 0.046*"available" + 0.023*"complaints" + 0.015*"continue" + 0.012*"capable" + 0.012*"bring" + 0.012*"command" + 0.012*"care" + 0.011*"career goals" + 0.010*"case incident"
2022-12-18 03:14:28,253 : INFO : topic #1 (0.500): 0.043*"campaign" + 0.020*"class learning" + 0.017*"assist paperwork" + 0.015*"child abuse" + 0.014*"child removal" + 0.012*"center military" + 0.012*"care" + 0.011*"advocate" + 0.011*"advertisment" + 0.010*"conduct check"
2022-12-18 03:14:28,254 : INFO : topic diff=0.001195, rho=0.213201
2022-12-18 03:14:28,286 : INFO : -5.699 per-word bound, 52.0 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:14:28,287 : INFO : PROGRESS: pass 21, at document #195/195
2022-12-18 03:14:28,310 : INFO : topic #0 (0.500): 0.052*"board" + 0.046*"available" + 0.023*"complaints" + 0.015*"continue" + 0.012*"capable" + 0.012*"bring" + 0.012*"care" + 0.012*"command" + 0.011*"career 

2022-12-18 03:14:28,832 : INFO : topic diff=0.000768, rho=0.176777
2022-12-18 03:14:28,863 : INFO : -5.697 per-word bound, 51.9 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:14:28,864 : INFO : PROGRESS: pass 31, at document #195/195
2022-12-18 03:14:28,884 : INFO : topic #0 (0.500): 0.052*"board" + 0.047*"available" + 0.023*"complaints" + 0.015*"continue" + 0.013*"care" + 0.012*"capable" + 0.012*"bring" + 0.012*"command" + 0.011*"career goals" + 0.010*"contact families"
2022-12-18 03:14:28,889 : INFO : topic #1 (0.500): 0.043*"campaign" + 0.020*"class learning" + 0.017*"assist paperwork" + 0.015*"child abuse" + 0.014*"child removal" + 0.011*"advocate" + 0.011*"center military" + 0.011*"advertisment" + 0.011*"care" + 0.010*"conduct check"
2022-12-18 03:14:28,891 : INFO : topic diff=0.000746, rho=0.174078
2022-12-18 03:14:28,926 : INFO : -5.697 per-word bound, 51.9 perplexity estimate based on a held-out corpus of 195 documents with 118 word

2022-12-18 03:14:29,463 : INFO : topic #0 (0.500): 0.052*"board" + 0.047*"available" + 0.023*"complaints" + 0.015*"continue" + 0.014*"care" + 0.013*"capable" + 0.012*"command" + 0.012*"bring" + 0.011*"career goals" + 0.010*"contact families"
2022-12-18 03:14:29,464 : INFO : topic #1 (0.500): 0.043*"campaign" + 0.020*"class learning" + 0.017*"assist paperwork" + 0.015*"child abuse" + 0.014*"child removal" + 0.011*"advocate" + 0.011*"advertisment" + 0.010*"conduct check" + 0.010*"center military" + 0.010*"care"
2022-12-18 03:14:29,465 : INFO : topic diff=0.000623, rho=0.152499
2022-12-18 03:14:29,501 : INFO : -5.691 per-word bound, 51.7 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:14:29,502 : INFO : PROGRESS: pass 42, at document #195/195
2022-12-18 03:14:29,520 : INFO : topic #0 (0.500): 0.052*"board" + 0.047*"available" + 0.023*"complaints" + 0.015*"continue" + 0.014*"care" + 0.013*"capable" + 0.012*"command" + 0.012*"bring" + 0.011*"care

2022-12-18 03:14:30,101 : INFO : topic diff=0.000500, rho=0.137361
2022-12-18 03:14:30,135 : INFO : -5.690 per-word bound, 51.6 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:14:30,136 : INFO : PROGRESS: pass 52, at document #195/195
2022-12-18 03:14:30,155 : INFO : topic #0 (0.500): 0.052*"board" + 0.047*"available" + 0.023*"complaints" + 0.015*"continue" + 0.014*"care" + 0.012*"capable" + 0.012*"command" + 0.012*"bring" + 0.011*"career goals" + 0.010*"center military"
2022-12-18 03:14:30,157 : INFO : topic #1 (0.500): 0.043*"campaign" + 0.020*"class learning" + 0.017*"assist paperwork" + 0.015*"child abuse" + 0.014*"child removal" + 0.011*"advocate" + 0.011*"advertisment" + 0.010*"aloud" + 0.010*"conduct check" + 0.010*"assist"
2022-12-18 03:14:30,158 : INFO : topic diff=0.000489, rho=0.136083
2022-12-18 03:14:30,190 : INFO : -5.690 per-word bound, 51.6 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12

2022-12-18 03:14:30,754 : INFO : topic #1 (0.500): 0.043*"campaign" + 0.020*"class learning" + 0.017*"assist paperwork" + 0.015*"child abuse" + 0.014*"child removal" + 0.011*"advocate" + 0.011*"advertisment" + 0.011*"aloud" + 0.010*"conduct check" + 0.010*"assist"
2022-12-18 03:14:30,754 : INFO : topic diff=0.000435, rho=0.125000
2022-12-18 03:14:30,789 : INFO : -5.689 per-word bound, 51.6 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:14:30,790 : INFO : PROGRESS: pass 63, at document #195/195
2022-12-18 03:14:30,814 : INFO : topic #0 (0.500): 0.051*"board" + 0.047*"available" + 0.022*"complaints" + 0.015*"continue" + 0.015*"care" + 0.012*"capable" + 0.012*"command" + 0.012*"bring" + 0.011*"center military" + 0.011*"career goals"
2022-12-18 03:14:30,815 : INFO : topic #1 (0.500): 0.043*"campaign" + 0.020*"class learning" + 0.017*"assist paperwork" + 0.015*"child abuse" + 0.014*"child removal" + 0.011*"advocate" + 0.011*"advertisment" + 0.01

2022-12-18 03:14:31,349 : INFO : PROGRESS: pass 73, at document #195/195
2022-12-18 03:14:31,372 : INFO : topic #0 (0.500): 0.051*"board" + 0.047*"available" + 0.022*"complaints" + 0.015*"care" + 0.015*"continue" + 0.013*"command" + 0.012*"capable" + 0.012*"bring" + 0.011*"center military" + 0.011*"career goals"
2022-12-18 03:14:31,377 : INFO : topic #1 (0.500): 0.044*"campaign" + 0.020*"class learning" + 0.017*"assist paperwork" + 0.015*"child abuse" + 0.014*"child removal" + 0.011*"advocate" + 0.011*"advertisment" + 0.011*"aloud" + 0.010*"conduct check" + 0.010*"assist"
2022-12-18 03:14:31,378 : INFO : topic diff=0.000494, rho=0.115470
2022-12-18 03:14:31,411 : INFO : -5.689 per-word bound, 51.6 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:14:31,412 : INFO : PROGRESS: pass 74, at document #195/195
2022-12-18 03:14:31,431 : INFO : topic #0 (0.500): 0.051*"board" + 0.047*"available" + 0.022*"complaints" + 0.016*"care" + 0.015*"continue" +

[(0,
  '0.051*"board" + 0.047*"available" + 0.022*"complaints" + 0.016*"care" + 0.015*"continue" + 0.013*"command" + 0.012*"capable" + 0.012*"bring" + 0.012*"center military" + 0.011*"career goals"'),
 (1,
  '0.044*"campaign" + 0.020*"class learning" + 0.017*"assist paperwork" + 0.015*"child abuse" + 0.014*"child removal" + 0.012*"advocate" + 0.011*"advertisment" + 0.011*"aloud" + 0.010*"conduct check" + 0.010*"assist"')]

In [93]:
#trying topic modeling with 3 topics
sparse_counts = scipy.sparse.csr_matrix(term_doc_2)
corpus_2 = matutils.Sparse2Corpus(sparse_counts)
id2word_2 = dict((v, k) for k, v in tv2.vocabulary_.items())
lda_2 = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word_2, passes=80)
lda_2.print_topics()

2022-12-18 03:15:32,175 : INFO : using symmetric alpha at 0.3333333333333333
2022-12-18 03:15:32,177 : INFO : using symmetric eta at 0.3333333333333333
2022-12-18 03:15:32,180 : INFO : using serial LDA version on this node
2022-12-18 03:15:32,181 : INFO : running online (multi-pass) LDA training, 3 topics, 80 passes over the supplied corpus of 195 documents, updating model once every 195 documents, evaluating perplexity every 195 documents, iterating 50x with a convergence threshold of 0.001000
2022-12-18 03:15:32,247 : INFO : -10.196 per-word bound, 1173.2 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:15:32,248 : INFO : PROGRESS: pass 0, at document #195/195
2022-12-18 03:15:32,318 : INFO : topic #0 (0.333): 0.055*"board" + 0.045*"campaign" + 0.018*"care" + 0.015*"assist paperwork" + 0.013*"ask" + 0.013*"available" + 0.012*"bring" + 0.010*"case" + 0.009*"brief" + 0.009*"ask families"
2022-12-18 03:15:32,320 : INFO : topic #1 (0.333): 0.02

2022-12-18 03:15:32,888 : INFO : topic #2 (0.333): 0.058*"available" + 0.031*"complaints" + 0.025*"class learning" + 0.021*"center military" + 0.021*"continue" + 0.020*"command" + 0.018*"child abuse" + 0.016*"child removal" + 0.013*"advocate" + 0.013*"conduct check"
2022-12-18 03:15:32,890 : INFO : topic diff=0.007187, rho=0.333333
2022-12-18 03:15:32,928 : INFO : -5.985 per-word bound, 63.3 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:15:32,928 : INFO : PROGRESS: pass 8, at document #195/195
2022-12-18 03:15:32,944 : INFO : topic #0 (0.333): 0.073*"board" + 0.056*"campaign" + 0.021*"assist paperwork" + 0.019*"care" + 0.017*"bring" + 0.014*"ask" + 0.013*"capable support" + 0.013*"case incident" + 0.013*"advertisment" + 0.013*"case"
2022-12-18 03:15:32,945 : INFO : topic #1 (0.333): 0.022*"capable" + 0.020*"career goals" + 0.016*"contact families" + 0.015*"board school" + 0.013*"contact" + 0.012*"base" + 0.012*"class" + 0.011*"check quarte

2022-12-18 03:15:33,473 : INFO : topic #2 (0.333): 0.064*"available" + 0.031*"complaints" + 0.025*"class learning" + 0.021*"center military" + 0.021*"continue" + 0.020*"command" + 0.018*"child abuse" + 0.016*"child removal" + 0.013*"advocate" + 0.013*"conduct check"
2022-12-18 03:15:33,473 : INFO : topic diff=0.002158, rho=0.242536
2022-12-18 03:15:33,514 : INFO : -5.955 per-word bound, 62.0 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:15:33,514 : INFO : PROGRESS: pass 16, at document #195/195
2022-12-18 03:15:33,538 : INFO : topic #0 (0.333): 0.077*"board" + 0.057*"campaign" + 0.021*"assist paperwork" + 0.018*"care" + 0.017*"bring" + 0.015*"capable support" + 0.014*"ask" + 0.013*"case incident" + 0.013*"advertisment" + 0.013*"case"
2022-12-18 03:15:33,539 : INFO : topic #1 (0.333): 0.023*"capable" + 0.020*"career goals" + 0.019*"contact families" + 0.015*"board school" + 0.014*"contact" + 0.013*"benefits" + 0.013*"base" + 0.012*"class" +

2022-12-18 03:15:33,989 : INFO : topic #2 (0.333): 0.066*"available" + 0.031*"complaints" + 0.025*"class learning" + 0.021*"center military" + 0.021*"continue" + 0.019*"command" + 0.018*"child abuse" + 0.016*"child removal" + 0.014*"care" + 0.013*"advocate"
2022-12-18 03:15:33,990 : INFO : topic diff=0.001635, rho=0.200000
2022-12-18 03:15:34,030 : INFO : -5.948 per-word bound, 61.7 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:15:34,031 : INFO : PROGRESS: pass 24, at document #195/195
2022-12-18 03:15:34,048 : INFO : topic #0 (0.333): 0.078*"board" + 0.057*"campaign" + 0.021*"assist paperwork" + 0.018*"bring" + 0.016*"care" + 0.015*"capable support" + 0.014*"ask" + 0.013*"advertisment" + 0.013*"case incident" + 0.013*"case"
2022-12-18 03:15:34,050 : INFO : topic #1 (0.333): 0.023*"capable" + 0.020*"career goals" + 0.019*"contact families" + 0.015*"board school" + 0.014*"benefits" + 0.014*"contact" + 0.013*"base" + 0.012*"class" + 0.011*"c

2022-12-18 03:15:34,521 : INFO : topic #2 (0.333): 0.066*"available" + 0.031*"complaints" + 0.024*"class learning" + 0.021*"center military" + 0.021*"continue" + 0.019*"command" + 0.018*"child abuse" + 0.016*"child removal" + 0.016*"care" + 0.013*"advocate"
2022-12-18 03:15:34,523 : INFO : topic diff=0.000921, rho=0.174078
2022-12-18 03:15:34,564 : INFO : -5.945 per-word bound, 61.6 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:15:34,564 : INFO : PROGRESS: pass 32, at document #195/195
2022-12-18 03:15:34,587 : INFO : topic #0 (0.333): 0.079*"board" + 0.058*"campaign" + 0.021*"assist paperwork" + 0.018*"bring" + 0.015*"capable support" + 0.015*"care" + 0.014*"ask" + 0.013*"advertisment" + 0.013*"case" + 0.013*"case incident"
2022-12-18 03:15:34,588 : INFO : topic #1 (0.333): 0.023*"capable" + 0.020*"career goals" + 0.019*"contact families" + 0.015*"board school" + 0.014*"benefits" + 0.014*"contact" + 0.013*"base" + 0.012*"class" + 0.011*"c

...

2022-12-18 03:15:35,970 : INFO : topic #2 (0.333): 0.066*"available" + 0.031*"complaints" + 0.024*"class learning" + 0.021*"center military" + 0.020*"care" + 0.020*"continue" + 0.019*"command" + 0.017*"child abuse" + 0.016*"child removal" + 0.013*"advocate"
2022-12-18 03:15:35,972 : INFO : topic diff=0.000727, rho=0.132453
2022-12-18 03:15:36,000 : INFO : -5.933 per-word bound, 61.1 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:15:36,001 : INFO : PROGRESS: pass 56, at document #195/195
2022-12-18 03:15:36,021 : INFO : topic #0 (0.333): 0.080*"board" + 0.058*"campaign" + 0.022*"assist paperwork" + 0.018*"bring" + 0.015*"capable support" + 0.014*"advertisment" + 0.014*"ask" + 0.013*"case" + 0.012*"case incident" + 0.012*"center"
2022-12-18 03:15:36,022 : INFO : topic #1 (0.333): 0.023*"capable" + 0.020*"career goals" + 0.019*"contact families" + 0.015*"board school" + 0.014*"benefits" + 0.014*"contact" + 0.013*"base" + 0.012*"class" + 0.011*

2022-12-18 03:15:36,427 : INFO : topic #2 (0.333): 0.066*"available" + 0.030*"complaints" + 0.024*"class learning" + 0.022*"care" + 0.021*"center military" + 0.020*"continue" + 0.019*"command" + 0.017*"child abuse" + 0.016*"child removal" + 0.013*"advocate"
2022-12-18 03:15:36,429 : INFO : topic diff=0.000410, rho=0.124035
2022-12-18 03:15:36,462 : INFO : -5.931 per-word bound, 61.0 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:15:36,463 : INFO : PROGRESS: pass 64, at document #195/195
2022-12-18 03:15:36,483 : INFO : topic #0 (0.333): 0.080*"board" + 0.059*"campaign" + 0.022*"assist paperwork" + 0.018*"bring" + 0.016*"capable support" + 0.014*"advertisment" + 0.014*"ask" + 0.014*"case" + 0.012*"center" + 0.012*"case incident"
2022-12-18 03:15:36,484 : INFO : topic #1 (0.333): 0.023*"capable" + 0.020*"career goals" + 0.019*"contact families" + 0.015*"board school" + 0.014*"benefits" + 0.014*"contact" + 0.013*"base" + 0.012*"class" + 0.011*

2022-12-18 03:15:36,888 : INFO : topic #2 (0.333): 0.066*"available" + 0.030*"complaints" + 0.024*"class learning" + 0.023*"care" + 0.021*"center military" + 0.020*"continue" + 0.019*"command" + 0.017*"child abuse" + 0.016*"child removal" + 0.013*"advocate"
2022-12-18 03:15:36,888 : INFO : topic diff=0.000296, rho=0.117041
2022-12-18 03:15:36,923 : INFO : -5.930 per-word bound, 61.0 perplexity estimate based on a held-out corpus of 195 documents with 118 words
2022-12-18 03:15:36,924 : INFO : PROGRESS: pass 72, at document #195/195
2022-12-18 03:15:36,949 : INFO : topic #0 (0.333): 0.080*"board" + 0.059*"campaign" + 0.022*"assist paperwork" + 0.018*"bring" + 0.016*"capable support" + 0.014*"advertisment" + 0.014*"case" + 0.013*"ask" + 0.012*"center" + 0.012*"case incident"
2022-12-18 03:15:36,950 : INFO : topic #1 (0.333): 0.023*"capable" + 0.020*"career goals" + 0.019*"contact families" + 0.015*"board school" + 0.014*"benefits" + 0.014*"contact" + 0.013*"base" + 0.012*"class" + 0.011*

2022-12-18 03:15:37,382 : INFO : topic #2 (0.333): 0.065*"available" + 0.030*"complaints" + 0.024*"care" + 0.024*"class learning" + 0.021*"center military" + 0.020*"continue" + 0.019*"command" + 0.017*"child abuse" + 0.016*"child removal" + 0.013*"advocate"
2022-12-18 03:15:37,383 : INFO : topic diff=0.000259, rho=0.111111
2022-12-18 03:15:37,384 : INFO : LdaModel lifecycle event {'msg': 'trained LdaModel(num_terms=307, num_topics=3, decay=0.5, chunksize=2000) in 5.20s', 'datetime': '2022-12-18T03:15:37.384266', 'gensim': '4.1.2', 'python': '3.8.13 (default, Mar 28 2022, 06:16:26) \n[Clang 12.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}
2022-12-18 03:15:37,385 : INFO : topic #0 (0.333): 0.080*"board" + 0.059*"campaign" + 0.022*"assist paperwork" + 0.018*"bring" + 0.016*"capable support" + 0.014*"advertisment" + 0.014*"case" + 0.013*"ask" + 0.012*"center" + 0.012*"case incident"
2022-12-18 03:15:37,386 : INFO : topic #1 (0.333): 0.023*"capable" + 0.020*"caree

[(0,
  '0.080*"board" + 0.059*"campaign" + 0.022*"assist paperwork" + 0.018*"bring" + 0.016*"capable support" + 0.014*"advertisment" + 0.014*"case" + 0.013*"ask" + 0.012*"center" + 0.012*"case incident"'),
 (1,
  '0.023*"capable" + 0.020*"career goals" + 0.019*"contact families" + 0.015*"board school" + 0.014*"benefits" + 0.014*"contact" + 0.013*"base" + 0.012*"class" + 0.011*"check quarterly" + 0.009*"clear"'),
 (2,
  '0.065*"available" + 0.030*"complaints" + 0.024*"care" + 0.024*"class learning" + 0.021*"center military" + 0.020*"continue" + 0.019*"command" + 0.017*"child abuse" + 0.016*"child removal" + 0.013*"advocate"')]